In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gekko

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gekko import GEKKO
import pandas as pd
import math
from pathlib import Path
import os

> ## **qsun Calculates the irradiation on a outer surface**
 
         SYNTAX: E=irrad(Dh,En,iday,LST,gamma,beta,rground)
         OUTPUT: global irradiation on a surface
         
Splitted irradiation

         E(1)= diffuse solar irradiation on an inclined surface
         E(2)= direct solar irradiation on an inclined surface
         E(3)= total solar irradiation on an inclined surface
         E(4)= total solar irradiation on a horizontal surface

INPUT:

        (scalar) Dh = diffuse horizontal irradiation[W/m2]
        (scalar) En = direct normal irradiation [W/m2]
        (scalar) t = time seconds after midnight 1 january
        (scalar) gamma = azimuth angle of the surface,
        east:gamma = -90, west:gamma = 90
        south:gamma = 0, north:gamma = 180
        (scalar) beta = inclination angle of the surface,
        horizontal: beta=0, vertical: beta=90
        default geographical position: De Bilt
        default ground reflectivity (albedo): 0.2
 
EXAMPLE: E=irrad(800,200,201,12,0,45)
ANSWER: E=1.0e+003 *
0.8569 0.1907 1.0759 0.9684

REF: Perez (zie Solar Energy volume 39 no. 3)

Adapted version from Eindhoven University of Technology: JvS feb 2002

In [ ]:
def qsun(tclim,
        Dh,
        En,
        gamma,
        beta,
        rground):
    #(scalar) iday = day of the year (1-365)
    #(scalar) LST = Local Standard time (0 - 24) [hour]

    
        iday = 1+math.floor(tclim/(24*3600))
        LST  = math.floor((tclim/3600) % 24)
        # L = Latitude [graden]
        L    = 52.1
        # LON = Local Longitude [graden] oost is positief
        LON  = 5.1
        # LSM = Local Standard time Meridian [graden] oost is positief
        LSM  = 15
        # rground = albedo
        # rground=0.2;
        r    = math.pi/180;
        L    = L*r
        beta = beta*r
        theta= 2*math.pi*(iday-1)/365.25
        el   = 4.901+0.033*math.sin(-0.031+theta)+theta
        # declination

        delta= math.asin(math.sin(23.442*r)*math.sin(el))
        q1   = math.tan(4.901+theta)
        q2   = math.cos(23.442*r)*math.tan(el)
        # equation of time

        ET   = (math.atan((q1-q2)/(q1*q2+1)))*4/r
        AST  = LST+ET/60-(4/60)*(LSM-LON)
        h    = (AST-12)*15*r
        # hai=sin(solar altitude)
        hai  = math.cos(L)*math.cos(delta)*math.cos(h)+math.sin(L)*math.sin(delta)
        
       
        E    = np.zeros((1,4))
        
        if hai>0:
            # salt=solar altitude

            salt= math.asin(hai)
            phi = math.acos((hai*math.sin(L)-math.sin(delta))/(math.cos(salt)*math.cos(L)))*np.sign(h)
            gam = phi-gamma*r;
            # cai=cos(teta)
            cai = math.cos(salt)*math.cos(abs(gam))*math.sin(beta)+hai*math.cos(beta)
            # teta = incident angle on the tilted surface
            teta= math.acos(cai)
            # salts=solar altitude for an inclined surface

            salts=math.pi/2-teta
            # Perez (zie Solar Energy volume 39 no. 3)
            # berekening van de diffuse straling op een schuin vlak
            # Approximatin of A and C, the solid angles occupied by the circumsolar region,
            # weighed by its average incidence on the slope and horizontal respectively.
            # In the expression of diffuse on inclined surface the quotient of A/C is
            # reduced to XIC/XIH. A=2*(1-cos(beta))*xic, C=2*(1-cos(beta))*xih
            # gecontroleerd okt 1996 martin de wit

            # alpha= the half-angle circumsolar region
            alpha=25*r

            if salts<-alpha:

                xic=0

            elif salts>alpha:

                xic=cai

            else:
                xic=0.5*(1+salts/alpha)*math.sin((salts+alpha)/2)


            if salt>alpha:

                xih=hai

            else:
                xih=math.sin((alpha+salt)/2)


            epsint=[1.056 ,1.253 ,1.586, 2.134, 3.23 ,5.98, 10.08 ,999999]
            f11acc=[-0.011 ,-0.038 ,0.166 ,0.419 ,0.710 ,0.857 ,0.734 ,0.421]
            f12acc=[0.748 ,1.115 ,0.909 ,0.646 ,0.025 ,-0.370 ,-0.073 ,-0.661]
            f13acc=[-0.080 ,-0.109 ,-0.179 ,-0.262 ,-0.290 ,-0.279 ,-0.228 ,0.097]
            f21acc=[-0.048 ,-0.023 ,0.062 ,0.140 ,0.243 ,0.267 ,0.231 ,0.119]
            f22acc=[0.073 ,0.106 ,-0.021 ,-0.167 ,-0.511 ,-0.792 ,-1.180 ,-2.125]
            f23acc=[-0.024 ,-0.037 ,-0.050 ,-0.042 ,-0.004 ,0.076 ,0.199 ,0.446]
            
            # determination of zet = solar zenith angle (pi/2 - solar altitude).
            
            zet=math.pi/2-salt
            
            # determination of inteps with eps

            inteps=0

            if Dh>0:

                eps=1+En/Dh
                inteps=7     # give big random number for starting point
                for i in range(len(epsint)):
                    if epsint[i]>=eps:
                        temp_i =i        
                        inteps=min(temp_i,inteps)
                #print(inteps)
                
                #inteps=min(i)

            # calculation of inverse relative air mass

            airmiv=hai

            if salt<10*r:
                airmiv=hai+0.15*(salt/r+3.885)**(-1.253)   #change ^ to **
            # calculation of extraterrestrial radiation

            Eon=1370*(1+0.033*math.cos(2*math.pi*(iday-3)/365))
            
            #delta is "the new sky brightness parameter"

            delta=Dh/(airmiv*Eon)

            # determination of the "new circumsolar brightness coefficient
            # (f1acc) and horizon brightness coefficient (f2acc)"
            
            f1acc=f11acc[inteps]+f12acc[inteps]*delta+f13acc[inteps]*zet
            f2acc=f21acc[inteps]+f22acc[inteps]*delta+f23acc[inteps]*zet
            
            # determination of the diffuse radiation on an inclined surface

            E[0,0]=Dh*(0.5*(1+math.cos(beta))*(1-f1acc)+f1acc*xic/xih+f2acc*math.sin(beta))

            if E[0,0]<0:

                E[0,0]=0

            # horizontal surfaces treated separately
            # beta=0 : surface facing up, beta=180(pi) : surface facing down
            #3/22/19 10:05 AM E:\NEN5060\qsun.m 4 of 4
            
            if beta>-0.0001 and beta<0.0001:

                E[0,0]=Dh

            if beta>(math.pi-0.0001) and beta<(math.pi+0.0001):

                E[0,0]=0
                
             # Isotropic sky
             # E(1)=0.5*(1+cos(beta))*Dh;

             # direct solar radiation on a surface

            E[0,1]=En*cai

            if E[0,1]<0.0:

                E[0,1]=0

            # the ground reflected component: assume isotropic
            # ground conditions.

            Eg=0.5*rground*(1-math.cos(beta))*(Dh+En*hai)
            
            # global irradiation

            E[0,3]=Dh+En*hai

             # total irradiation on an inclined surface
            E[0,2]=E[0,0] + E[0,1] + Eg
            
        return E[0,2]

> ### Change working directory to input data file

In [ ]:
print(Path.cwd())
#print(os.listdir("../kaggle/input"))
parent_dir = os.chdir('/kaggle/')
data_dir = Path.cwd() /'input'
#output_dir = Path.cwd()/'working'/'submit'
NENdata_path = data_dir/'NEN5060-2018.xlsx'
xls = pd.ExcelFile(NENdata_path)    

In [ ]:
xls.sheet_names  # Check sheet names

In [ ]:
# select sheet k=1 by NEN default
k=1

> Exchange_NEN5060
>  
    Exchange of NEN5060 data in climate files for Python
    Exchange of NEN5060_2018 data in Excel
    for irradiation on S, SW, W, NW, N, NE, E, SE and horizontal and Toutdoor
    Irradiation can be used for solar irradiation on windows
    %Matlab version September 17th 2018 by Arie Taal THUAS (The Hague University of Applied Sciences)
    %Python version 28/05/2020 by Trung Nguyen HAN University of Applied Sciences

In [ ]:
rground=0  #ground reflection is ignored

for k in range(4):
    
    if k==1:
        
        # this file is part of NEN 5060 20018
        NUM = pd.read_excel(xls,'nen5060 - energie') # this file is part of NEN 5060 20018
        
    elif k==2:
        
        NUM = pd.read_excel(xls,'ontwerp 1%')
        
    elif k==3:
        
        NUM = pd.read_excel(xls,'ontwerp 5%')
        
    #Convert data frame to array
    to_array=NUM.to_numpy()
    to_array=np.delete(to_array, 0, 0)
    
    t=(np.array(list(range(1,8761)))-1)*3600
    dom=to_array[:,2] # day of month
    hod=to_array[:,3] # hour of day
    qglob_hor=to_array[:,4]
    qdiff_hor=to_array[:,5]
    qdir_hor=to_array[:,6]
    qdir_nor=to_array[:,7]
    Toutdoor=to_array[:,8]/10
    phioutdoor=to_array[:,9]
    xoutdoor=to_array[:,10]/10
    pdamp=to_array[:,11]
    vwind=to_array[:,12]/10  #% at 10 m height
    dirwind=to_array[:,13]
    cloud=to_array[:,14]/10
    rain=to_array[:,15]/10
    
    #w, h = 9, 8760;
    #E=[[0 for x in range(w)] for y in range(h)]
    E= np.zeros((8760,9))
    for j in range(9):
        if j<9:
            gamma=45*(j-1)
            beta=90
        else:
            
            gamma=90
            beta=0
        
        for i in range(8759):
            
            #E[i][j]=qsun(t[i],qdiff_hor[i],qdir_nor[i],gamma,beta,rground)
            E[i,j]=qsun(t[i],qdiff_hor[i],qdir_nor[i],gamma,beta,rground)
    myarray = np.asarray(E)       
    #myarray = np.delete(myarray,8759, 0)        
    qsunS=np.vstack((t,myarray[:,0]))
    qsunSW=np.vstack((t,myarray[:,1]))
    qsunW=np.vstack((t,myarray[:,2]))
    qsunNW=np.vstack((t,myarray[:,3]))
    qsunNW=np.vstack((t,myarray[:,4]))
    qsunNE=np.vstack((t,myarray[:,5]))
    qsunE=np.vstack((t,myarray[:,6]))
    qsunSE=np.vstack((t,myarray[:,7]))
    qsunhor=np.vstack((t,myarray[:,8]))
    Tout=np.vstack((t,Toutdoor))
    phiout=np.vstack((t,phioutdoor))
    xout=np.vstack((t,xoutdoor))
    pout=np.vstack((t,pdamp))
    vout=np.vstack((t,vwind))
    dirvout=np.vstack((t,dirwind))
    cloudout=np.vstack((t,cloud))
    rainout=np.vstack((t,rain))
    

In [ ]:
#Np.newaxis convert an 1D array to either a row vector or a column vector.
#check np.reshape 

a = Tout[np.newaxis]            #create 2D array from 1D array with 2 elements
a = a.T                         #Transpose array

#numrows = len(input)    # 3 rows in your example
#numcols = len(input[0]) # 2 columns in your example

plt.plot(a[:,0],a[:,1])

In [ ]:
import gc
gc.collect() # collect garbages

### Define input parameters

In [ ]:
print(qsunS[1])
print(qsunSW[0])
Qsolar_irradiation = sum(qsunS + qsunSW)
print(Qsolar_irradiation)